In [51]:
#CODE BASED ON: https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613

In [52]:
# !pip install -q transformers

In [53]:
import torch
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [54]:
df = pd.read_csv('UpdatedResumeDataSet.csv')

In [55]:
df.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [56]:
df['Category'].value_counts()

Java Developer               84
Testing                      70
DevOps Engineer              55
Python Developer             48
Web Designing                45
HR                           44
Hadoop                       42
Blockchain                   40
ETL Developer                40
Operations Manager           40
Data Science                 40
Sales                        40
Mechanical Engineer          40
Arts                         36
Database                     33
Electrical Engineering       30
Health and fitness           30
PMO                          30
Business Analyst             28
DotNet Developer             28
Automation Testing           26
Network Security Engineer    25
SAP Developer                24
Civil Engineer               24
Advocate                     20
Name: Category, dtype: int64

In [57]:
possible_labels = df.Category.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict

{'Data Science': 0,
 'HR': 1,
 'Advocate': 2,
 'Arts': 3,
 'Web Designing': 4,
 'Mechanical Engineer': 5,
 'Sales': 6,
 'Health and fitness': 7,
 'Civil Engineer': 8,
 'Java Developer': 9,
 'Business Analyst': 10,
 'SAP Developer': 11,
 'Automation Testing': 12,
 'Electrical Engineering': 13,
 'Operations Manager': 14,
 'Python Developer': 15,
 'DevOps Engineer': 16,
 'Network Security Engineer': 17,
 'PMO': 18,
 'Database': 19,
 'Hadoop': 20,
 'ETL Developer': 21,
 'DotNet Developer': 22,
 'Blockchain': 23,
 'Testing': 24}

In [58]:
df['label'] = df.Category.replace(label_dict)

In [59]:
df.head()

,Category,Resume,label
0,Data Science,Skills * Programming Languages: Python (pandas...,0
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,0
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",0
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,0
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",0


In [60]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size=0.5, 
                                                  random_state=42, 
                                                  stratify=df.label.values)

In [61]:
df['data_type'] = ['not_set']*df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [62]:
df.groupby(['Category', 'label', 'Resume']).count()

data_type
Category      label Resume                                                       
Advocate      2     Education Details \r\n B.Com, LL.B.,   Universi...          2
                    Education Details \r\n LLB.   Dibrugarh Univers...          2
                    Education Details \r\nNovember 2016 to January ...          2
                    Good grasping quality and skillful work Educati...          2
                    QUALIFICATION: Introduction to Computer EXTRAEd...          2
...                                                                           ...
Testing       24    â Willingness to accept the challenges. â P...         10
Web Designing 4     Education Details \r\n B.C.A Bachelor Computer ...          9
                    Education Details \r\nJanuary 2016 B.Sc. Inform...          9
                    IT SKILLS Languages: C (Basic), JAVA (Basic) We...          9
                    Technical Skills Web Technologies: Angular JS, ...         18

[166 rows x 1 columns]

In [63]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)

In [64]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].Resume.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].Resume.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    max_length=512, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2336: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [65]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [66]:
len(dataset_train), len(dataset_val)

(481, 481)

In [67]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [68]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [69]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [70]:
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [71]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

def accuracy_count(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    count = 0
    count_t = 0
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        count += len(y_preds[y_preds==label])
        count_t += len(y_true)
    print(f'Correct: {count}')
    print(f'Total: {count_t}\n')     
    return count, count_t

In [72]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [73]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [74]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [75]:
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/161 [00:00<?, ?it/s]


Epoch 1
Training loss: 3.087625959645147
Validation loss: 2.768173586507762
F1 Score (Weighted): 0.14197988426917674


Epoch 2:   0%|          | 0/161 [00:00<?, ?it/s]


Epoch 2
Training loss: 2.5555868918851297
Validation loss: 2.250955013014515
F1 Score (Weighted): 0.5322145323785036


...


Epoch 3
Training loss: 2.0794198053964177
Validation loss: 1.8455201220808561
F1 Score (Weighted): 0.7975994942656546


Epoch 4:   0%|          | 0/161 [00:00<?, ?it/s]


Epoch 4
Training loss: 1.785295253214629
Validation loss: 1.6188831047982162
F1 Score (Weighted): 0.865603231765434


Epoch 5:   0%|          | 0/161 [00:00<?, ?it/s]


Epoch 5
Training loss: 1.615055935723441
Validation loss: 1.539924937745799
F1 Score (Weighted): 0.8710595439421351


In [76]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [77]:
model.load_state_dict(torch.load('finetuned_BERT_epoch_5.model', map_location=torch.device('cuda')))

<All keys matched successfully>

In [78]:
_, predictions, true_vals = evaluate(dataloader_validation)

In [79]:
accuracy_per_class(predictions, true_vals)

Class: Data Science
Accuracy: 20/20

Class: HR
Accuracy: 22/22

Class: Advocate
Accuracy: 10/10

Class: Arts
Accuracy: 18/18

Class: Web Designing
Accuracy: 23/23

Class: Mechanical Engineer
Accuracy: 13/20

Class: Sales
Accuracy: 20/20

Class: Health and fitness
Accuracy: 15/15

Class: Civil Engineer
Accuracy: 12/12

Class: Java Developer
Accuracy: 42/42

Class: Business Analyst
Accuracy: 8/14

Class: SAP Developer
Accuracy: 0/12

Class: Automation Testing
Accuracy: 3/13

Class: Electrical Engineering
Accuracy: 15/15

Class: Operations Manager
Accuracy: 20/20

Class: Python Developer
Accuracy: 24/24

Class: DevOps Engineer
Accuracy: 25/27

Class: Network Security Engineer
Accuracy: 13/13

Class: PMO
Accuracy: 15/15

Class: Database
Accuracy: 16/16

Class: Hadoop
Accuracy: 21/21

Class: ETL Developer
Accuracy: 14/20

Class: DotNet Developer
Accuracy: 2/14

Class: Blockchain
Accuracy: 20/20

Class: Testing
Accuracy: 35/35



In [80]:
x,y = accuracy_count(predictions, true_vals)
x/y


Correct: 426
Total: 481



0.8856548856548857

In [81]:
#70/30 split data
# Class: Data Science
# Accuracy: 12/12

# Class: HR
# Accuracy: 13/13

# Class: Advocate
# Accuracy: 6/6

# Class: Arts
# Accuracy: 11/11

# Class: Web Designing
# Accuracy: 14/14

# Class: Mechanical Engineer
# Accuracy: 12/12

# Class: Sales
# Accuracy: 12/12

# Class: Health and fitness
# Accuracy: 9/9

# Class: Civil Engineer
# Accuracy: 7/7

# Class: Java Developer
# Accuracy: 25/25

# Class: Business Analyst
# Accuracy: 8/8

# Class: SAP Developer
# Accuracy: 7/7

# Class: Automation Testing
# Accuracy: 5/8

# Class: Electrical Engineering
# Accuracy: 9/9

# Class: Operations Manager
# Accuracy: 12/12

# Class: Python Developer
# Accuracy: 14/14

# Class: DevOps Engineer
# Accuracy: 16/17

# Class: Network Security Engineer
# Accuracy: 8/8

# Class: PMO
# Accuracy: 9/9

# Class: Database
# Accuracy: 10/10

# Class: Hadoop
# Accuracy: 13/13

# Class: ETL Developer
# Accuracy: 12/12

# Class: DotNet Developer
# Accuracy: 1/8

# Class: Blockchain
# Accuracy: 12/12

# Class: Testing
# Accuracy: 21/21

# Correct: 278
# Total: 289
# 0.9619377162629758

# Wrong:
# Class: Automation Testing
# Accuracy: 5/8
# Class: DevOps Engineer
# Accuracy: 16/17
# Class: DotNet Developer
# Accuracy: 1/8

In [ ]:
#50/50 split data
# Class: Data Science
# Accuracy: 20/20

# Class: HR
# Accuracy: 22/22

# Class: Advocate
# Accuracy: 10/10

# Class: Arts
# Accuracy: 18/18

# Class: Web Designing
# Accuracy: 23/23

# Class: Mechanical Engineer
# Accuracy: 13/20

# Class: Sales
# Accuracy: 20/20

# Class: Health and fitness
# Accuracy: 15/15

# Class: Civil Engineer
# Accuracy: 12/12

# Class: Java Developer
# Accuracy: 42/42

# Class: Business Analyst
# Accuracy: 8/14

# Class: SAP Developer
# Accuracy: 0/12

# Class: Automation Testing
# Accuracy: 3/13

# Class: Electrical Engineering
# Accuracy: 15/15

# Class: Operations Manager
# Accuracy: 20/20

# Class: Python Developer
# Accuracy: 24/24

# Class: DevOps Engineer
# Accuracy: 25/27

# Class: Network Security Engineer
# Accuracy: 13/13

# Class: PMO
# Accuracy: 15/15

# Class: Database
# Accuracy: 16/16

# Class: Hadoop
# Accuracy: 21/21

# Class: ETL Developer
# Accuracy: 14/20

# Class: DotNet Developer
# Accuracy: 2/14

# Class: Blockchain
# Accuracy: 20/20

# Class: Testing
# Accuracy: 35/35

# Correct: 426
# Total: 481
# 0.8856548856548857

# Class: Mechanical Engineer
# Accuracy: 13/20
# Class: Business Analyst
# Accuracy: 8/14
# Class: SAP Developer
# Accuracy: 0/12
# Class: Automation Testing
# Accuracy: 3/13
# Class: DevOps Engineer
# Accuracy: 25/27
# Class: ETL Developer
# Accuracy: 14/20
# Class: DotNet Developer
# Accuracy: 2/14